In [1]:
%pwd
!pip install requests

### Filtering I (BM25 Retrieval)

In [2]:
import sys
import json
import requests
import time
start_time = time.time()
################################################################################
# Filtering
################################################################################
pretty = lambda x : json.dumps(x, indent=2, sort_keys=True)
solr_select = 'http://localhost:8983/solr/depcc-small/select?q='
#train_path  = 'data/04-hyper/train.jsonl'
#train_path  = 'data/02-acl-arc/train.jsonl'
train_path  = 'data/07-imdb/train.jsonl'
MAX_TR = 2000
num_tr = 0
with open(train_path, 'r') as train_file:
    json_lines = []
    lines = train_file.readlines()
    for line in lines:
        if num_tr == MAX_TR:
            break
        j = json.loads(line)
        json_lines.append(j)
        num_tr += 1
N = len(json_lines)
print(N)

# Query with training example
j = json_lines[0]
query = j['text'].replace(' ', '+')
#query = ""
#with open('data/02-acl-arc/lda_union.txt', 'r') as file:
#    query = file.read()
print(query)
#print(len(query.split()))
#sys.exit()
rp_retrieval = requests.get(solr_select + query).json()
cc_docs = (rp_retrieval['response']['docs'])
print('Number of retrieved documents: %d' % len(cc_docs))
cc_doc0 = json.loads(cc_docs[0]['_src_'])
cc_doc100 = ""
for i in range(10):
    cc_doc100 += json.loads(cc_docs[i]['_src_'])['text']
print(cc_doc100)
print("--- %s seconds ---" % (time.time() - start_time))
################################################################################

2000
Dr.+Seuss+would+sure+be+mad+right+now+if+he+was+alive.+Cat+in+the+Hat+proves+to+show+how+movie+productions+can+take+a+classic+story+and+turn+it+into+a+mindless+pile+of+goop.+We+have+Mike+Myers+as+the+infamous+Cat+in+the+Hat,+big+mistake!+Myers+proves+he+can't+act+in+this+film.+He+acts+like+a+prissy+show+girl+with+a+thousand+tricks+up+his+sleeve.+The+kids+in+this+movie+are+all+right,+somewhere+in+between+the+lines+of+dull+and+annoying.+The+story+is+just+like+the+original+with+a+couple+of+tweaks+and+like+most+movies+based+on+other+stories,+never+tweak+with+the+original+story!+Bringing+in+the+evil+neighbor+Quin+was+a+bad+idea.+He+is+a+stupid+villain+that+would+never+get+anywhere+in+life.This+movie+is+like+a+rejected+comic+strip+from+the+newspaper+if+you+think+about+it.+The+film+sure+does+look+tacky!+Sure+there+are+a+funny+adult+jokes+like+where+the+cat+cuts+of+his+tail+and+the+censor+goes+off+before+he+says+a+naughty+word,+mildly+funny.+At+least+the+Grinch+had+spunk,+and+the+film+was

### Segmentation by 100 Tokens(Documents -> Passages)

In [3]:
################################################################################
# Segmentation
################################################################################
#val_url = cc_doc0['url']
#print(val_url)
#val_s3 = cc_doc0['s3']
#print(val_s3)
#cc_words = cc_doc0['text'].split()
'''


cc_words = cc_doc100.split()

cc_psgs = []
num_psgs = len(cc_words) // 100
print('Number of passages: %d ' % num_psgs)
for i in range(num_psgs):
    start = i * 100
    end   = -1 if (i == num_psgs-1) else (i+1) * 100
    psg = ' '.join(cc_words[start:end])
    dict = {'doc_id' : val_url, 'doc_text'  : psg,  'title': ''  }
    cc_psgs.append(dict)

print('Passage 0')
print(cc_psgs[0]['doc_text'])
'''

"\n\n\ncc_words = cc_doc100.split()\n\ncc_psgs = []\nnum_psgs = len(cc_words) // 100\nprint('Number of passages: %d ' % num_psgs)\nfor i in range(num_psgs):\n    start = i * 100\n    end   = -1 if (i == num_psgs-1) else (i+1) * 100\n    psg = ' '.join(cc_words[start:end])\n    dict = {'doc_id' : val_url, 'doc_text'  : psg,  'title': ''  }\n    cc_psgs.append(dict)\n\nprint('Passage 0')\nprint(cc_psgs[0]['doc_text'])\n"

### Segmentation by sentences(Documents -> Passages)

In [4]:
#!python -m spacy download en_core_web_sm
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["parser"])
nlp.enable_pipe("senter")
doc = nlp(cc_doc100)
cc_psgs = []
psg = ''
num_tokens = 0
for sent in doc.sents:
    if num_tokens < 100:
        psg += sent.text
        num_tokens += len(sent)
    else:
#        print(num_tokens)
        cc_psgs.append({'doc_id' : '', 'doc_text'  : psg,  'title': ''  })
        num_tokens = 0
        psg = ''
#print(len(cc_psgs))

### Encoding Preparation

In [5]:
################################################################################
# Encoding Preparation
################################################################################

import csv
import subprocess

# Prepare cc_sample.tsv
f_out = 'emb/cc_sample.tsv'
with open(f_out, 'w') as output_file:
    dw = csv.DictWriter(output_file, cc_psgs[0].keys(), delimiter='\t')
    for psg in cc_psgs:
        dw.writerow(psg)
print('CC sample')
print(cc_psgs[0]['doc_text'])  
print()
# Prepare train_sample.tsv
num_train = len(json_lines)
train_psgs = []
for i in range(num_train):
    train_dict = {'doc_id': str(i), 'doc_text': json_lines[i]['text'], 'title': ''}
    train_psgs.append(train_dict)
    
print('train sample')
print(train_psgs[0]['doc_text'])

f_train_out = 'emb/train_sample.tsv'
with open(f_train_out, 'w') as output_file:
    dw = csv.DictWriter(output_file, train_psgs[0].keys(), delimiter='\t')
    for tp in train_psgs:
        dw.writerow(tp)

CC sample
Studio : Universal Year : 2003 Rated : PG Film Length : 78 minutes Aspect Ratio : Anamorphic Widescreen ( 1.85:1 ) Captions : English Subtitles : French , and Spanish Audio : English - Dolby Digital 5.1 ; French and Spanish - Dolby Digital 5.1 .Please forgive my abruptness .I was going to think of a clever rhyme to start this review out with , but after watching this movie , I am so mad that I want to dive right in .Bo Welch 's adaptation of Dr. Seuss ' beloved The Cat in the Hat is a shameful butchering of one of the most popular children 's books ever written - so much so that it is almost not even worth the effort to rip it .

train sample
Dr. Seuss would sure be mad right now if he was alive. Cat in the Hat proves to show how movie productions can take a classic story and turn it into a mindless pile of goop. We have Mike Myers as the infamous Cat in the Hat, big mistake! Myers proves he can't act in this film. He acts like a prissy show girl with a thousand tricks up his

### Encode CC

In [6]:
# Encode cc_sample (tsv->embedding)
!bash emb/generate_embedding.sh cc_sample

Initialized host node123 as d.rank -1 on device=cuda, n_gpu=8, world size=1
16-bits training: False 
Reading saved model from /mnt/nfs/work1/696ds-s21/hhwang/DPR/data/checkpoint/retriever/multiset/bert-base-encoder.cp
model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = bert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_bert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 256
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /mnt/nfs/work1/696ds-s21/hhwang/da/emb/cc_sample.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_low

### Encode Train Sample

In [7]:
# Encode train_sample (tsv-> embedding)
!bash emb/generate_embedding.sh train_sample

Initialized host node123 as d.rank -1 on device=cuda, n_gpu=8, world size=1
16-bits training: False 
Reading saved model from /mnt/nfs/work1/696ds-s21/hhwang/DPR/data/checkpoint/retriever/multiset/bert-base-encoder.cp
model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
Overriding args parameter value from checkpoint state. Param = do_lower_case, value = True
Overriding args parameter value from checkpoint state. Param = pretrained_model_cfg, value = bert-base-uncased
Overriding args parameter value from checkpoint state. Param = encoder_model_type, value = hf_bert
Overriding args parameter value from checkpoint state. Param = sequence_length, value = 256
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /mnt/nfs/work1/696ds-s21/hhwang/da/emb/train_sample.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_

In [8]:
MAX_TR_PSGS = len(train_psgs)
MAX_CC_PSGS = len(cc_psgs)
print(MAX_TR_PSGS)
print(MAX_CC_PSGS)

2000
466


### Nearest Neighbors

In [9]:
################################################################################
# Nearest Neighbor (FAISS)
################################################################################
#print(emb[0])
import numpy as np
import faiss

# Read CC embeddings (DATABASE)
cc_embeddings = np.load('emb/cc_sample_0.pkl', allow_pickle=True)
print(cc_embeddings[0][1].shape)  # Dimension of the embedding
nb = len(cc_embeddings) # database size
d = cc_embeddings[0][1].size
print(nb,d)
xb = np.zeros((nb,d), dtype='float32')
for i in range(nb):
    xb[i] = cc_embeddings[i][1]
print('Number of CC passages: %d' % nb)

# Read train embeddings (QUERY)
train_embeddings = np.load('emb/train_sample_0.pkl', allow_pickle=True)
print(train_embeddings[0][1].shape)  # Dimension of the embedding
nq = len(train_embeddings[:10]) # database size
d = train_embeddings[0][1].size
print(nq,d)
xq = np.zeros((nq,d), dtype='float32')
for i in range(nq):
    xq[i] = train_embeddings[i][1]

print('Number of train passages: %d' % nq)
print(xq)


index = faiss.IndexFlatL2(d)   # build the index

print('trained? %r' % index.is_trained)
index.add(xb)                  # add vectors to the index
print('Total number of indexed CC passages: ', index.ntotal)
print()
print('Using an indentical CC set')
k = nb                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print('================================================================')
print('4 nearest neighbors')
print(I)
print()
print('distances(sanity check)')
print(D)
print()


print('===============================================================')
print('Using the query(train set)')
D, I = index.search(xq, k)     # actual search
print('4 nearest neighbors')
print(I[:5])                   # neighbors of the 5 first queries
print('\ndistances')
print(D)
print()



(768,)
466 768
Number of CC passages: 466
(768,)
10 768
Number of train passages: 10
[[-0.34161064  0.12626015  0.330658   ...  0.01783237  0.8322366
   0.5383658 ]
 [-0.22916777  0.13893938 -0.04974905 ...  0.31890947  0.5730853
  -0.44882116]
 [-0.3348288  -0.10564958  0.4709152  ...  0.5587872   0.3357489
   0.07439476]
 ...
 [ 0.24057896  0.35483631 -0.17971207 ... -0.07679785 -0.29069793
  -0.09099212]
 [-0.13248383  0.28628424 -0.03393117 ... -0.37087503  0.04048529
  -0.2180164 ]
 [ 0.10268944 -0.02661577  0.00835894 ... -0.27195162  0.0931703
   0.0613623 ]]
trained? True
Total number of indexed CC passages:  466

Using an indentical CC set
4 nearest neighbors
[[  0   6  10 ... 179 147 399]
 [  1  17 450 ... 364 147 399]
 [  2  13 189 ... 384 295 399]
 [  3   5  16 ...  70 182 399]
 [  4 132 117 ... 392  18 399]]

distances(sanity check)
[[  0.        47.70364   52.353226 ... 135.96915  142.73923  147.24966 ]
 [  0.        57.716187  62.315414 ... 137.71939  142.71222  145.8507

In [10]:
print('Train passage')
print(train_psgs[300]['doc_text'])
print()
print('CLOSEST passages in CC:')

for i in range(4):
    print('-------------------------------------------------------------')
    print('Closest %d' % i)
    closest = I[0][i]
    print(cc_psgs[closest]['doc_text'])

print('------------------------------------------------------------')
print('...')

for i in range(MAX_CC_PSGS-4, MAX_CC_PSGS):
    print('-------------------------------------------------------------')
    print('Farthest %d' % i)
    closest = I[0][i]
    print(cc_psgs[closest]['doc_text'])


Train passage
I ran across this several years ago while channel surfing on a Sunday afternoon. Though it was obviously a cheesy TV movie from the 70s, the direction and score were well done enough that it grabbed my attention, and indeed I was hooked and had to watch it through to the end. I recently got the opportunity to buy a foreign DVD of this film (oops, didn't notice a domestic one had finally come out a couple months prior), and was very pleased to be able to watch it again (and in its entirety).I don't wholly understand the phenomenon, but somehow the 70s seem to have a lock on horror movies that are actually scary. The decades prior to the 70s produced some beautifully shot films and the bulk of our enduring horror icons, but are they actually scary? No, not very. Likewise in the years since the 70s we've gotten horror movies that are cooler, more exciting, have much better production values and sophisticated special effects, are more fun, funnier, have effective "jump" momen